In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, t, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Inspiration and Research Objectives

The USDA Center for Nutrition Policy and Promotion recommends a very simple daily diet intake guideline: 30% grains, 40% vegetables, 10% fruits, and 20% protein, but are we really eating in the healthy eating style recommended by these food divisions and balances. What can we do to protect our loved ones, those who sacrifice for us, and ourselves from this pandemic?
 

# Data Collection and Gathering

About this file
This dataset includes percentage of fat, food quantity, energy, protein intake from different types of food in countries around the world. The last couple of columns also includes counts of obesity, undernourished, and COVID-19 cases as percentages of the total population for comparison purposes.

In [ ]:
import numpy as np
import pandas as pd

import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.impute import KNNImputer
import warnings
warnings.filterwarnings('ignore')

In [ ]:
fat = pd.read_csv('../input/covid19-healthy-diet-dataset/Fat_Supply_Quantity_Data.csv')
fat.head()

In [ ]:
quantity = pd.read_csv('../input/covid19-healthy-diet-dataset/Food_Supply_Quantity_kg_Data.csv')
quantity.head()

In [ ]:
kcal = pd.read_csv('../input/covid19-healthy-diet-dataset/Food_Supply_kcal_Data.csv')
kcal.head()

In [ ]:
protein = pd.read_csv('../input/covid19-healthy-diet-dataset/Protein_Supply_Quantity_Data.csv')
protein.head()

In [ ]:
supply = pd.read_csv('../input/covid19-healthy-diet-dataset/Supply_Food_Data_Descriptions.csv')
supply.head()

# Exploratory Data Analysis

In [ ]:
fat.isnull().sum()

In [ ]:
missing_col = [c for c in fat.columns if fat[c].isnull().sum() != 0]
missing_col

In [ ]:
#impute missing data
def impute_missing(df):
    #transform data from object variable to numerical variable
    df['Undernourished'] = df['Undernourished'].map(lambda x: '2.4' if x == '<2.5' else x).astype('float64')

    col = list(df.columns)
    col.remove('Unit (all except Population)') #remove '%' unit
    col.remove('Country') #object variable
    # nan_col = [c for c in col if df[c].isnull().sum() > 0]
    # not_nan_col = [c for c in col if c not in nan_col]


    imputer = KNNImputer(n_neighbors=3)
    imp_data = imputer.fit_transform(df[col])
    new = pd.DataFrame(imp_data, columns = col)

    return pd.concat([df[['Country']], new], axis=1) #concat 'Country' to data frame then return
    

In [ ]:
#apply impute missing data function
fat = impute_missing(fat)
quantity = impute_missing(quantity)
kcal = impute_missing(kcal)
protein = impute_missing(protein)

In [ ]:
fig, ax = plt.subplots(figsize=(10, 5))

df = fat[['Country', 'Confirmed', 'Deaths', 'Recovered', 'Active']]
df.sort_values(by='Confirmed', ascending=False, inplace=True)
sns.barplot(x="Confirmed", y="Country", data=df[:20]);

In [ ]:
fig, ax = plt.subplots(figsize=(10, 5))

df = fat[['Country', 'Confirmed', 'Deaths', 'Recovered', 'Active']]
df.sort_values(by='Active', ascending=False, inplace=True)
sns.barplot(x="Active", y="Country", data=df[:20]);

In [ ]:
fig, ax = plt.subplots(figsize=(10, 5))

df = fat[['Country', 'Confirmed', 'Deaths', 'Recovered', 'Active']]
df.sort_values(by='Recovered', ascending=False, inplace=True)
sns.barplot(x="Recovered", y="Country", data=df[:20]);

In [ ]:
fig, ax = plt.subplots(figsize=(10, 5))

df = fat[['Country', 'Confirmed', 'Deaths', 'Recovered', 'Active']]
df.sort_values(by='Deaths', ascending=False, inplace=True)
sns.barplot(x="Deaths", y="Country", data=df[:20]);

In [ ]:
sns.scatterplot(data=fat[fat['Population'] < 6*1e8], x="Population", y="Confirmed")

In [ ]:
fig, ax = plt.subplots(figsize=(20, 15))

mask = np.triu(np.ones_like(fat.corr(), dtype=np.bool))
mask = mask[1:, :-1]
corr = fat.corr().iloc[1:,:-1]

sns.heatmap(corr, mask=mask, annot=True, fmt=".2f", cmap='Blues',
           vmin=-1, vmax=1, cbar_kws={"shrink": .8})

In [ ]:
# get variable which the highest 12 variable
fat[abs(fat.corr()['Confirmed']).sort_values(ascending=False)[:12].index].corr()

In [ ]:
fig, ax = plt.subplots(figsize=(10, 10))

# get variable which the highest 12 variable
mask = np.triu(np.ones_like(fat[abs(fat.corr()['Confirmed']).sort_values(ascending=False)[:12].index].corr(), dtype=np.bool))
mask = mask[1:, :-1]
corr = fat[abs(fat.corr()['Confirmed']).sort_values(ascending=False)[:12].index].corr().iloc[1:,:-1]

sns.heatmap(corr, mask=mask, annot=True, fmt=".2f", cmap='Blues',linewidths = 1, square= True,
           vmin=-1, vmax=1, cbar_kws={"shrink": .8})

plt.title("Correlation of percentage of fat from different type of foods", fontsize = 13)
# plt.xlabel("Type of foods", fontsize = 13)
# plt.ylabel("Type of foods", fontsize = 13)
plt.show()

In [ ]:
fat.hist(figsize=(15,15));

In [ ]:
fat.columns

In [ ]:
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score

# features = abs(fat.corr()['Confirmed']).sort_values(ascending=False)[:12].index.to_list()
features = ['Alcoholic Beverages', 'Animal Products', 'Animal fats',
       'Aquatic Products, Other', 'Cereals - Excluding Beer', 'Eggs',
       'Fish, Seafood', 'Fruits - Excluding Wine', 'Meat', 'Miscellaneous',
       'Milk - Excluding Butter', 'Offals', 'Oilcrops', 'Pulses', 'Spices',
       'Starchy Roots', 'Stimulants', 'Sugar Crops', 'Sugar & Sweeteners',
       'Treenuts', 'Vegetal Products', 'Vegetable Oils', 'Vegetables',
       'Obesity', 'Undernourished', 'Deaths', 'Recovered', 'Population']


X = fat[features]
y = fat['Deaths']


inertia = []
for i in range(2,11):
    
    # intialise kmeans
    k = KMeans(n_clusters=i, max_iter=100, random_state=0)
    k.fit(X)
    
    inertia.append(k.inertia_)
    
    cluster_labels = k.labels_+1
    
    # silhouette score
    silhouette_avg = silhouette_score(X, cluster_labels)
    print("For n_clusters={}, the silhouette score is {:.4f}".format(i, silhouette_avg))

# Visualize elbow plot and choose the no. of cluster that make a conner or line in this case we choose 5 group.
plt.figure(figsize=(12,6))
sns.lineplot(x=[i for i in range(2,11)], y=inertia)
plt.scatter(3,inertia[1], s = 50, c = 'red', marker='D')
plt.title('The Elbow Method')
plt.xlabel('No. of Clusters')
plt.ylabel('Inertia');
plt.show()

In [ ]:
kmeans = KMeans(n_clusters=3, random_state=0).fit(X)
# kmeans.predict(X)
X['Group'] = kmeans.labels_+1
# df = pd.concat([X, fat[['Deaths']]], axis=1)
# df
# # kmeans.cluster_centers_[:,1]

fig, ax = plt.subplots(figsize=(10, 10))
sns.boxplot(x="Group", y="Recovered", data=X)

In [ ]:
# col = [fat.columns.map(lambda x: 'fat_'+x)]
# fat.columns = col

# col = [quantity.columns.map(lambda x: 'quantity_'+x)]
# quantity.columns = col

# col = [kcal.columns.map(lambda x: 'kcal_'+x)]
# kcal.columns = col

# col = [protein.columns.map(lambda x: 'protein_'+x)]
# protein.columns = col

# Model

In [ ]:
from sklearn.linear_model import LassoCV
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.linear_model import LinearRegression, LassoCV, RidgeCV
from sklearn.metrics import mean_squared_error, r2_score

features = ['Alcoholic Beverages', 'Animal Products', 'Animal fats',
       'Aquatic Products, Other', 'Cereals - Excluding Beer', 'Eggs',
       'Fish, Seafood', 'Fruits - Excluding Wine', 'Meat', 'Miscellaneous',
       'Milk - Excluding Butter', 'Offals', 'Oilcrops', 'Pulses', 'Spices',
       'Starchy Roots', 'Stimulants', 'Sugar Crops', 'Sugar & Sweeteners',
       'Treenuts', 'Vegetal Products', 'Vegetable Oils', 'Vegetables',
       'Obesity', 'Undernourished']

X = fat[features]
y = fat['Confirmed']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)

print('X_train shape',X_train.shape)
print('X_test shape',X_test.shape)
print('y_train shape',y_train.shape)
print('y_test shape',y_test.shape)

print('\n')
lasso = LassoCV(alphas = np.linspace(0.1,10,100))
lasso.fit(X_train, y_train)
print('R2 of Train', lasso.score(X_train, y_train))
print('R2 of Test', lasso.score(X_test, y_test))

lasso_rmse_cross_val = -cross_val_score(lasso, X_train, y_train, scoring='neg_root_mean_squared_error').mean()
print('RMSE for Unseen Data', lasso_rmse_cross_val)
print('RMSE for Test data', np.sqrt(mean_squared_error(y_test, lasso.predict(X_test))))

In [ ]:
# fea_imp = [{n,c} for n,c in zip(features, lasso.coef_) if c != 0]
# df = pd.DataFrame(fea_imp, columns=['Coefficient', 'Feature Names']).sort_values(by = 'Coefficient', ascending=False)
# df

fea_imp = [(n,c) for n,c in zip(features, lasso.coef_) if c != 0]
df = pd.DataFrame(fea_imp, columns=['Features Name', 'Coefficient']).sort_values(by = 'Coefficient', ascending=False)
df

In [ ]:
fig, ax = plt.subplots(figsize=(15,5))
# plt.figsize=(15,5)

sns.barplot(y=df['Coefficient'],x=df['Features Name'])
# plt.setp(ax.get_xticklabels(), rotation=90)
ax.set_xticklabels(ax.get_xticklabels(), rotation=90);

# pd.DataFrame(lasso.coef_, index = features).plot(kind='bar', figsize=(15,5));  #same viz

In [ ]:
sns.pairplot(fat, x_vars=df['Features Name'], y_vars=['Confirmed'], diag_kind =None);

In [ ]:
import statsmodels.api as sm
#https://www.statsmodels.org/devel/examples/notebooks/generated/ols.html

#Add constant because stat model need constant
X_train_addconst = sm.add_constant(X_train)
model = sm.OLS(y_train, X_train_addconst)
results = model.fit()
print(results.summary())

# Feature Engineering

In [ ]:
from sklearn.preprocessing import PolynomialFeatures

poly = PolynomialFeatures(degree=2, include_bias=False)
X_train_poly = poly.fit_transform(X_train)
X_test_poly = poly.transform(X_test)

X_train_poly = pd.DataFrame(X_train_poly, columns = poly.get_feature_names(X_train.columns))
X_train_poly

In [ ]:
lasso = LassoCV(alphas = np.linspace(0.1,10,100))

lasso.fit(X_train_poly, y_train)
print('R2 of Train', lasso.score(X_train_poly, y_train))
print('R2 of Test', lasso.score(X_test_poly, y_test))

lasso_rmse_cross_val = -cross_val_score(lasso, X_train_poly, y_train, scoring='neg_root_mean_squared_error').mean()
print('RMSE for Unseen Data', lasso_rmse_cross_val)
print('RMSE for Test data', np.sqrt(mean_squared_error(y_test, lasso.predict(X_test_poly))))

In [ ]:
fea_imp = [(n,c) for n,c in zip(X_train_poly.columns, lasso.coef_) if c != 0]
df = pd.DataFrame(fea_imp, columns=['Feature Names', 'Coefficient']).sort_values(by = 'Coefficient', ascending=False)
df

fig, ax = plt.subplots(figsize=(12,5))
sns.barplot(y=df['Coefficient'], x=df['Feature Names'])
ax.set_xticklabels(ax.get_xticklabels(), rotation=90);